In [6]:
#!pip install matplotlib
import numpy as np
import pandas as pd
#from sklearn.cluster import DBSCAN
#from pykalman import KalmanFilter
import matplotlib.pyplot as plt
from scipy.optimize import linear_sum_assignment
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.cm import ScalarMappable
from scipy.spatial.distance import cdist
import time
import csv
from collections import namedtuple

import sys
sys.path.append('build')  # Add the 'build' directory to the Python path

import dbscan_module  # Import the shared object file

In [7]:
def plot_kalman_update_point(cluster_points, cluster_center):
    # Assuming cluster_points is a 2D numpy array with x and y coordinates
    #print(cluster_points[-5:])
    num_points = cluster_points.shape[0]
    # Generate a range of colors that go from light to dark
    colors = np.linspace(0.3, 1, num_points)  # Adjust 0.3 to 1 for darker shades
    
    # Plot the cluster points with varying darkness
    scatter = plt.scatter(cluster_points[:, 0], cluster_points[:, 1], c=colors, cmap='Greys')
    
    # Plot the cluster center
    plt.scatter(cluster_center[0], cluster_center[1], color='red', marker='x', label='Cluster Center')
    
    # Add a colorbar, which will automatically use the 'Greys' colormap
    cbar = plt.colorbar(scatter)
    cbar.set_label('Point Intensity (later points darker)')
    
    plt.show()

def plot_dbscan_output(df, labels):
    #print("###############################")
    #print(unique_labels)
    global global_counter
    global_counter +=1
    print("DBSCAN counter: ",global_counter)
    fig, axs = plt.subplots(2, 2, figsize=(10, 10))
    axs[0,0].scatter(df["PeakTime"], df["El."], marker='o', c=labels)
    axs[0,1].scatter(df["PeakTime"], df["Az."], marker='o', c=labels)
    axs[1,0].scatter(df["El."], df["Az."], marker='o', c=labels)

    inds = labels > -1
    axs[1,1].scatter(df["El."][inds], df["Az."][inds], marker='o', c=labels[inds])
    # Extract x and y axis limits
    x_limits = axs[1, 1].get_xlim()
    y_limits = axs[1, 1].get_ylim()
    axs[1,1].scatter(df["El."], df["Az."], marker='o', c=labels)
    axs[1,1].set_xlim(x_limits)
    axs[1,1].set_ylim(y_limits)
    plt.show()



def PrintInfo(distance_matrix, associations, unassigned_clusters):
    print("#########################################")
    print("distance matrix")
    print(distance_matrix)
    print(f"associations: {associations}")
    print(f"unassigned_clusters: {unassigned_clusters}")
    for c in unassigned_clusters:
        print("Initializing new filter for cluster: ", c)




In [9]:
"""
This is a program that performs multi-target tracking. 
Initially, clustering (DBSCAN) is performed on a batch of samples to estimate the number of targets. 
A kalman filter is then initialized for each cluster (i.e. target).
New observations are used to either update an existing kalman filter's state or ignored as noise.
Periodically, batches of observations are used to perform clustering to update the number of targets. 
Each cluster is either associated with an existing kalman filter or used to initialize a new kalman filter.
If a kalman filter is not assocaited with a cluster for a specified number of clustering iterations, the filter is destroyed.
"""
global_counter = 0



# Load the data as per your existing code
"""
column_names = [
    "PeakTime", "Energy", "El.", "Az.", "TDOA12", "TDOA13", "TDOA14",
    "TDOA23", "TDOA24", "TDOA34", "Xcorr12", "Xcorr13", "Xcorr14",
    "Xcorr23", "Xcorr24", "Xcorr34"
]
"""
column_names = [
    "PeakTime", "Amplitude", "DOA_x", "DOA_y", "DOA_z", "TDOA12", "TDOA13", "TDOA14",
    "TDOA23", "TDOA24", "TDOA34", "Xcorr12", "Xcorr13", "Xcorr14",
    "Xcorr23", "Xcorr24", "Xcorr34"
]

# Replace with your actual data path
path_doa = "../../C/deployment_files/2024-8-29-18-37-36-480140_detection"
path_doa = "../../C/deployment_files/2024-9-25-13-5-15-419096_detection"
path_doa = "../../C/deployment_files/2024-10-1-15-19-31-643613_detection"
path_doa = "../../C/deployment_files/2024-10-1-15-55-32-804596_detection" #track154
#path_doa = "../2024-9-12-9-36-14-51349_detection"
path_doa = "../../C/deployment_files/new3.txt"
path_doa = "../../../listener_program/tracks/231105_090101_000000_tracker_132"

#df = pd.read_csv(path_doa, delim_whitespace=True, names=column_names, skiprows=1)
df = pd.read_csv(path_doa, sep=',', names=column_names, skiprows=1)

# Instantiate the tracker
#tracker = Tracker(eps=3, min_samples=15, missed_update_threshold=4)
#tracker = dbscan_module.Tracker(3, 15, 4)
#print('Before tracker')
tracker = dbscan_module.Tracker(0.04, 15, 4)
#print('After tracker')
# Main processing loop (outside the class)
#time_intervals = np.linspace(df["usec_since_Unix_Start"].min(), df["usec_since_Unix_Start"].max(), num=20)
# Convert 'PeakTime' column to datetime if needed; assuming the timestamp is in microseconds since epoch
timestamps = pd.to_datetime(df['PeakTime'], unit='us')

# Determine the start and end time based on the data
start_time = np.array(timestamps).min()
end_time = np.array(timestamps).max()

# Create 60-second intervals from start_time to end_time
# Create 60-second intervals starting from start_time until an interval end time exceeds end_time
time_intervals = [start_time]
while time_intervals[-1] + pd.Timedelta(seconds=30) <= end_time:
    time_intervals.append(time_intervals[-1] + pd.Timedelta(seconds=30))

grouped_time_intervals = []

# Iterate through each interval and extract observations
cluster_runtimes = 0
for ind in range(len(time_intervals) - 1):
    # Get start and end times for the current interval
    interval_start = time_intervals[ind]
    interval_end = time_intervals[ind + 1]

    print(interval_start,interval_end)
    
    # Filter observations within the current interval
    observations = df[
        (timestamps > interval_start) &
        (timestamps <= interval_end)
    ]

    # If there are no data points in this interval, skip to the next iteration
    if observations.empty:
        continue
    #print("hello")
    # Run DBSCAN on even intervals
    if ind % 2 == 0:
        #print("here", ind)
         
        if ind != 0:
            for _, row in observations.iterrows():
                #observation = [row["El."], row["Az."]]
                observation = [row["DOA_x"], row["DOA_y"], row["DOA_z"]]
                observation = np.squeeze(observation)
                tracker.update_kalman_filters_continuous(observation, int(row['PeakTime']))
                last_row = row
        #print("ind", ind)
        #print(time_intervals[ind])

        grouped_time_intervals.append([observations['PeakTime'].min(), observations['PeakTime'].max()])
        
        X = observations[["DOA_x", "DOA_y", "DOA_z"]].values.astype(np.float32)
        #print("Length:")
        print("length: ", len(X))
        time_diff_seconds = (time_intervals[ind] - time_intervals[ind +1]) / 1e6  # 1e6 to convert microseconds to seconds
        print(time_diff_seconds)
        bef_clust = time.time()
        tracker.process_batch(X) # Handle batch updates
        af_clust = time.time()
        runtime= af_clust - bef_clust
        cluster_runtimes += runtime
        #k = k+1
    else:
        # For odd intervals, handle individual sample updates
        for _, row in observations.iterrows():
            observation = [row["DOA_x"], row["DOA_y"], row["DOA_z"]]
            observation = np.squeeze(observation)
            tracker.update_kalman_filters_continuous(observation, int(row['PeakTime']))
            last_row = row

print("sleeping....")
time.sleep(10)
observation = [last_row["DOA_x"], last_row["DOA_y"], last_row["DOA_z"]]
observation = np.squeeze(observation)
tracker.update_kalman_filters_continuous(observation, int(last_row['PeakTime']))
print("done....")
print(cluster_runtimes)

2023-11-05T09:01:29.619200000 2023-11-05 09:01:59.619200
length:  21
-1 days +23:59:59.999970
2023-11-05 09:01:59.619200 2023-11-05 09:02:29.619200
2023-11-05 09:02:29.619200 2023-11-05 09:02:59.619200
length:  42
-1 days +23:59:59.999970
2023-11-05 09:02:59.619200 2023-11-05 09:03:29.619200
2023-11-05 09:03:29.619200 2023-11-05 09:03:59.619200
length:  71
-1 days +23:59:59.999970
2023-11-05 09:03:59.619200 2023-11-05 09:04:29.619200
2023-11-05 09:04:29.619200 2023-11-05 09:04:59.619200
length:  57
-1 days +23:59:59.999970
2023-11-05 09:04:59.619200 2023-11-05 09:05:29.619200
2023-11-05 09:05:29.619200 2023-11-05 09:05:59.619200
length:  45
-1 days +23:59:59.999970
2023-11-05 09:05:59.619200 2023-11-05 09:06:29.619200
2023-11-05 09:06:29.619200 2023-11-05 09:06:59.619200
length:  46
-1 days +23:59:59.999970
2023-11-05 09:06:59.619200 2023-11-05 09:07:29.619200
2023-11-05 09:07:29.619200 2023-11-05 09:07:59.619200
length:  100
-1 days +23:59:59.999970
2023-11-05 09:07:59.619200 2023-11-